In [1]:
import os
%pwd
os.chdir("../")

In [2]:
%pwd

'e:\\Datasets\\Hafeez_DataSets\\chicken-Disease-Classification'

In [3]:
import tensorflow as tf

In [4]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [16]:
from dataclasses import dataclass
import pathlib as Path
from cnnClassifier.constants import *

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int
    

from cnnClassifier.utils.common import read_yaml, create_directories, save_json
#CONFIG_FILE_PATH = Path("config/config.yaml")  # Ensure this is correctly defined
#PARAMS_FILE_PATH = Path("config/params.yaml")  # Ensure this is correctly defined

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config['artifacts_root']])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chicken-fecal-images",
            all_params= self.params,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE

        )
        return eval_config    

from urllib.parse import urlparse
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        
    def _valid_generartor(self):
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split =0.30 
        )
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[: 1],
            batch_size =self.config.params_batch_size,
            interpolation = "bilinear"
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self._valid_generartor = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False
            **dataflow_kwargs
        )
        
    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generartor()
        self.score = model.evaluate(self._valid_generartor)
        
    def save_score(self):
        scores = {"loss" : self.score[0], "accuracy": self.score[1]}
        save_json(path = Path("scores.json"), data = scores)

try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
    
except Exception as e:
    raise e
            

AttributeError: 'EvaluationConfig' object has no attribute 'training_date'